In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/acoes/busca-avancada/')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[4]/div/div[1]/div[2]/a/span').click()
time.sleep(2)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'

os.remove ('/Users/scheiner/Documents/GitHub/StatusInvest/stocks.csv')

dest = '/Users/scheiner/Documents/GitHub/StatusInvest/stocks.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [3]:
### Tratamento dos dados importados:

df = pd.read_csv("stocks.csv", sep=';', decimal=',', thousands='.', encoding="utf-8")
    
df['P/L_IDEAL'] = df[' VPA'] / df[' LPA']
df['UP/DOWNSIDE'] = (df[' VPA'] / df['PRECO'])-1
    
valuation = df.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]

valuation.fillna(value=0, inplace=True)
valuation.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
valuation.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
valuation.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
valuation = valuation.loc[valuation['DY'] >= 6].loc[
    valuation['P/L'] >= 0.01].loc[valuation['P/L'] < 15].loc[
    valuation['ROE'] >= 6].loc[
    valuation['EV/EBIT'] >= 0.01].loc[valuation['EV/EBIT'] <= 15].loc[
    valuation['LIQ.CORR'] >= 0.5]


In [4]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = valuation['TICKER']
lista_stocks = ticker.to_list()

lista_indicadores_stocks = []
for stocks in lista_stocks:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/{stocks}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        PRECO = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[1]/div/div[1]/strong').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[1]/strong').text
        DY = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[4]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[2]/div/span[2]/b').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[3]/div/div[3]/div/div/div/a/strong').text
        
    

        #Criando dict
        dicionario = {  "stocks": stocks,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        lista_indicadores_stocks.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()


In [7]:
### Preparando os dados e Dataframe

list_stocks = pd.DataFrame.from_dict(lista_indicadores_stocks)

list_stocks = list_stocks.replace('-', '')
list_stocks = list_stocks.replace('-%', '')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.lstrip('R$ ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.lstrip('R$ ')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.strip('- ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.strip('- ')
list_stocks['VAL_12M'] = (list_stocks['VAL_12M']).str.rstrip('%')
list_stocks['VAL_MES'] = (list_stocks['VAL_MES']).str.rstrip('%')
list_stocks.to_csv('list_stocks.csv')

list_stocks = pd.read_csv("list_stocks.csv", sep=',', decimal=',', thousands='.', encoding="utf-8" )

list_stocks = list_stocks.drop(['Unnamed: 0'],axis=1)
list_stocks.fillna(value=0, inplace=True)


valuation.rename(columns={'TICKER': "TICKERS" }, inplace=True)

list_stocks.rename(columns={'stocks': "TICKERS" }, inplace=True)

investing = pd.merge(valuation, list_stocks, on='TICKERS')

value_invest = investing[['TICKERS', 'PRECO', 'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S',  'VAL_12M', 
                       ' VPA', 'DY', ' LPA', 'P/L' ,'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 'P/VP', 'EV/EBIT', 
                       'LIQ.CORR', 'MARG.LIQ', 'DIV.LIQ/PL', 'SEGMENTO']]

### PEGANDO O INDICE IBOVESPA NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

t = get_tickers('IBOV')
ibov = pd.DataFrame(t)
ibov['IBOVESPA'] = 'IBOV'
ibov.rename(columns={0: "TICKERS" }, inplace=True)
ibov = ibov


### PEGANDO O INDICE SMALL CAPS NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

u = get_tickers('SMLL')

smll = pd.DataFrame(u)
smll['SMALLCAP'] = 'SMLL'
smll.rename(columns={0: "TICKERS" }, inplace=True)
smll = smll

### PEGANDO O INDICE SMALL IDIV NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

v = get_tickers('IDIV')

idiv = pd.DataFrame(v)
idiv['DIVIDENDOS'] = 'IDIV'
idiv.rename(columns={0: "TICKERS" }, inplace=True)
idiv = idiv

value_invest = pd.merge(value_invest, ibov, on='TICKERS', how = 'left').fillna(0)

value_invest = pd.merge(value_invest, smll, on='TICKERS', how = 'left').fillna(0)

value_invest = pd.merge(value_invest, idiv, on='TICKERS', how = 'left').fillna(0)

value_invest['IBOVESPA'] = (value_invest['IBOVESPA']).str.strip('0')
value_invest['SMALLCAP'] = (value_invest['SMALLCAP']).str.strip('0')
value_invest['DIVIDENDOS'] = (value_invest['DIVIDENDOS']).str.strip('0')

value_invest.fillna('', inplace=True)

value_invest = value_invest.loc[value_invest['DY'] >= 6].loc[
    value_invest['P/L'] >= 0.01].loc[value_invest['P/L'] < 15].loc[
    value_invest['ROE'] >= 6].loc[
    value_invest['EV/EBIT'] >= 0.01].loc[value_invest['EV/EBIT'] <= 15].loc[
    value_invest['LIQ.CORR'] >= 0.5].loc[
    value_invest['UP/DOWNSIDE'] >= 0.1]#.loc[
#     # value_invest['IBOVESPA'] == 'IBOV'].loc[
#     # value_invest['SMALLCAP'] == 'SMLL'].loc[
#     # value_invest['DIVIDENDOS'] == 'IDIV']

value_invest

OSError: [Errno 50] Network is down